In [35]:
import subprocess
from itertools import product
from math import prod as mul
from random import randrange

import numpy as np
import pandas as pd
from multiprocess import Pool

## Мета лабораторної роботи

Практично ознайомитися із сучасним методом криптоаналізу блокових шифрів, набути навички у дослідженні стійкості
блокових шифрів до диференціального криптоаналізу.

## Постановка задачі

|                       Постановка задачі                        |    Зроблено    |
|:--------------------------------------------------------------:|:--------------:|
|                Реалізувати функції шифру Хейса                 | $ \checkmark $ |
| Пошук високоімовірних п’ятираундових диференціалів шифру Хейса | $ \checkmark $ |
|        Реалізувати атаку на сьомий раундовий ключ Хейса        | $ \checkmark $ |

## Хід роботи / Опис труднощів
На початку практикуму разом думали яку мову краще вибрати для виконання лабораторної роботи і зійшлися на Python, через те, що
на ній просто буде легше реалізувати практикум. В основному проблеми виникали із:
* нестрогою типізацією Python, іноді не було зрозуміло де який тип і як їх можна створювати;
* розпаралелюванням коду для швидшого виконання. Виникла проблема із простим розумінням як це можна зробити у Python, але, після прочитання інструкцій, стало зрозуміло;
* розумінням того як треба рахувати значення ймовірностей для атаки, так як у схемі деякі кроки були пропущенні;
* розумінням самих позначень, на схемі, а саме букв у деяких випадках;
* розумінням того на який ключ і скільки ітерацій треба для використання методу "гілок та границь". Усі незрозумілості розвіялися із тим моментом, як почасли робити пошук ключів.

## Опис методу пошуку високоімовірних диференціалів, обрані порогові значення імовірностей (із обґрунтуванням вибору)

todo: explain main search algorithm
todo: explain why we have taken such probs


Варіант 5

S = (F, 8, E, 9, 7, 2, 0, D, C, 6, 1, 5, B, 4, 3, A)



In [36]:
s_block = [0xF, 0x8, 0xE, 0x9, 0x7, 0x2, 0x0, 0xD, 0xC, 0x6, 0x1, 0x5, 0xB, 0x4, 0x3, 0xA]
s_block_inverse = [0x6, 0xA, 0x5, 0xE, 0xD, 0xB, 0x9, 0x4, 0x1, 0x3, 0xF, 0xC, 0x8, 0x7, 0x2, 0x0]
ENC = True
DEC = False
MAX_16BIT_NUM = (1 << 16) - 1
VARIANT = 8


def get_hex(l):
    return '[{}]'.format(', '.join(hex(x) for x in l))


def get_hex_tuple_0(l):
    return '[{}]'.format(', '.join((hex(x[0])) for x in l))


class heys:

    def heys_round(self, n, key=0):
        ct = n ^ key
        ct = self.substitute(ct, s_block)
        ct = self.mix_words(ct)
        return ct

    # r = (s_1(y_1), s_2(y_2), ... , s_n(y_n)) 
    def substitute(self, n, enc):
        s = s_block if enc == True else s_block_inverse
        r = 0
        for i in range(4):
            r |= s[(n >> (i * 4)) & 15] << (i * 4)
        return r

    # i-тий біт j-того фрагменту стає j-тим бітом i-того фрагменту.
    def mix_words(self, n):
        r = 0
        for j in range(4):
            for i in range(4):
                r |= (n >> (4 * j + i) & 1) << (4 * i + j)
        return r


Створюємо таблицю диференціалів

$$ f(x \circ a) \bullet (f(x))^{-1} = b $$
так як у нас лише операції $\oplus$, тож воно претворюється на 
$$ f(x \oplus a) \oplus f(x) = b $$


таблицю диференціальних імовірностей S-блоку вашого варіанту;

In [37]:
heys_obj = heys()
# таблиця диференціалів для перестановки шифру Хейса
diff_substitution = np.zeros((16, 16))

for alpha in range(16):
    for beta in range(16):
        for x in range(16):
            diff_substitution[alpha][beta] += heys_obj.substitute(x ^ alpha, ENC) == heys_obj.substitute(x, ENC) ^ beta

diff_substitution /= 16

pd.DataFrame(diff_substitution)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.0,0.000,0.000,0.000,0.125,0.125,0.000,0.250,0.000,0.125,0.125,0.000,0.000,0.125,0.000,0.125
2,0.0,0.250,0.000,0.125,0.000,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.000,0.125,0.125,0.125
3,0.0,0.125,0.125,0.000,0.000,0.000,0.250,0.250,0.000,0.125,0.125,0.000,0.000,0.000,0.000,0.000
4,0.0,0.000,0.250,0.000,0.125,0.000,0.000,0.125,0.125,0.000,0.125,0.000,0.000,0.000,0.125,0.125
5,0.0,0.000,0.000,0.125,0.000,0.000,0.125,0.000,0.125,0.125,0.000,0.125,0.000,0.250,0.000,0.125
6,0.0,0.125,0.000,0.000,0.000,0.125,0.000,0.000,0.000,0.125,0.125,0.125,0.125,0.000,0.000,0.250
7,0.0,0.000,0.125,0.000,0.000,0.250,0.125,0.000,0.125,0.000,0.000,0.000,0.125,0.000,0.250,0.000
8,0.0,0.000,0.000,0.250,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.000,0.250,0.000,0.125,0.125
9,0.0,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.125,0.250,0.125,0.125,0.000,0.000,0.125,0.000


In [38]:
PROBABILITY_THRESHOLD = 0.0008
local_diff_table = {}


# betas
def get_new_betas(alpha):
    # obtain raw alpha values from one big Alpha
    alpha_list = [(alpha >> (4 * i)) & 15 for i in range(4)]
    # obtain non-null betas from s block diff table
    non_null_betas = []
    for i, alpha in enumerate(alpha_list):
        # filter betas with non-zero entries
        betas = [beta for beta in range(16) if diff_substitution[alpha][beta] != 0]
        non_null_betas.append(betas)

    betas = {}
    # generate all possible combinations that can appear in resulted beta
    for (i, beta_list) in enumerate(product(*non_null_betas, repeat=1)):
        beta = sum([beta_list[i] << (i * 4) for i in range(4)])
        betas[beta] = mul([diff_substitution[alpha_list[i]][beta_list[i]] for i in range(4)])

    return betas


def differential_search(alpha):
    # shared between workers variable
    global local_diff_table

    # Г_{t-1}(alpha) (Г spells like 'g')
    g_prev = {alpha: 1}
    # like in rust -- 1..=5
    for i in range(1, 6):
        g_current = {}
        # misuse of 'beta' naming (by now we are iterating over last alphas right before calculation of probabilities)
        for beta, p_i in g_prev.items():

            if beta not in local_diff_table:
                local_diff_table[beta] = get_new_betas(beta)

            for gamma in local_diff_table[beta]:
                tmp_prob = p_i * local_diff_table[beta][gamma]
                if gamma not in g_current:
                    g_current[gamma] = tmp_prob
                else:
                    g_current[gamma] += tmp_prob

        # Г_{t}(alpha)
        g_new_filtered = {}
        #check "bounds" and write to new list mixed words (aka gammas are becoming new alphas)
        for gamma in g_current.keys():
            if g_current[gamma] > PROBABILITY_THRESHOLD:
                g_new_filtered[heys_obj.mix_words(gamma)] = g_current[gamma]

        g_prev = g_new_filtered

    return (alpha, g_prev)

Ініціалізуємо pool воркерів
[1](https://superfastpython.com/multiprocessing-pool-python/#What_Are_Python_Processes) 
[2](https://superfastpython.com/multiprocessing-pool-shared-global-variables/)


знайдені за допомогою методу «гілок та границь» диференціали для кожного
раунду шифрування та їх імовірності (якщо перелік відповідних диференціалів завеликий,
дозволяється обмежитись певною вибіркою значень);

In [39]:
%%time

def prepare_alphas():
    alfas = []
    for i in range(4):
        for j in range(1, 16):
            alfas.append(j << 4 * i)
    return alfas


def init_worker(shared_diff_table):
    global local_diff_table
    local_diff_table = shared_diff_table


with Pool(initializer=init_worker, initargs=(local_diff_table,)) as pool:
    candidates = list(
        pool.map(
            differential_search,
            prepare_alphas()
        )
    )

print('Possible candidates after differential search: ')
list(candidates)

Possible candidates after differential search: 
CPU times: user 264 ms, sys: 50.8 ms, total: 315 ms
Wall time: 3.22 s


[(1, {}),
 (2, {}),
 (3, {}),
 (4, {546: 0.0009160041809082031}),
 (5, {}),
 (6, {}),
 (7, {}),
 (8, {}),
 (9, {}),
 (10, {}),
 (11, {}),
 (12, {}),
 (13, {}),
 (14, {}),
 (15, {}),
 (16, {}),
 (32, {}),
 (48, {}),
 (64, {1092: 0.000835418701171875}),
 (80, {}),
 (96, {}),
 (112, {}),
 (128, {}),
 (144, {}),
 (160, {}),
 (176, {}),
 (192, {}),
 (208, {}),
 (224, {}),
 (240, {}),
 (256, {}),
 (512, {}),
 (768, {273: 0.0009918212890625}),
 (1024,
  {273: 0.00206756591796875,
   4369: 0.00133514404296875,
   2184: 0.0014445781707763672,
   34952: 0.001047372817993164,
   17: 0.0010986328125,
   4352: 0.0008087158203125,
   4368: 0.00084686279296875,
   136: 0.0008516311645507812,
   546: 0.0009028911590576172}),
 (1280, {}),
 (1536, {}),
 (1792, {273: 0.0010528564453125}),
 (2048, {}),
 (2304, {}),
 (2560,
  {273: 0.001422882080078125,
   4369: 0.0009613037109375,
   2184: 0.0009412765502929688,
   17: 0.00081634521484375}),
 (2816,
  {273: 0.00080108642578125,
   4368: 0.0008010864257812

Витягуємо із можливих кандидатів список можливих ключів

In [40]:
filtered_candidates = []
for alpha, betas in list(candidates):
    for beta, prob in betas.items():
        filtered_candidates.append(((alpha, beta), prob))
filtered_candidates = sorted(filtered_candidates, key=lambda x: x[1], reverse=True)
filtered_candidates

[((1024, 273), 0.00206756591796875),
 ((3840, 273), 0.0015869140625),
 ((1024, 2184), 0.0014445781707763672),
 ((2560, 273), 0.001422882080078125),
 ((3072, 4368), 0.001346588134765625),
 ((3072, 4369), 0.001346588134765625),
 ((1024, 4369), 0.00133514404296875),
 ((3840, 2184), 0.0012450218200683594),
 ((3072, 273), 0.00124359130859375),
 ((1024, 17), 0.0010986328125),
 ((3072, 17), 0.0010833740234375),
 ((3584, 273), 0.001068115234375),
 ((3072, 4096), 0.001064300537109375),
 ((1792, 273), 0.0010528564453125),
 ((1024, 34952), 0.001047372817993164),
 ((3072, 1), 0.00103759765625),
 ((3840, 4369), 0.00102996826171875),
 ((768, 273), 0.0009918212890625),
 ((3072, 16), 0.000988006591796875),
 ((2560, 4369), 0.0009613037109375),
 ((3072, 2184), 0.0009605884552001953),
 ((3072, 34944), 0.0009605884552001953),
 ((3072, 34952), 0.0009605884552001953),
 ((2560, 2184), 0.0009412765502929688),
 ((4, 546), 0.0009160041809082031),
 ((1024, 546), 0.0009028911590576172),
 ((3328, 273), 0.000877380

знайдений в ході диференціальної атаки ключ останнього раунду шифрування
тестової програми, із зазначенням кількості шифртекстів, що були потрібні для
знаходження;

In [41]:
%%time

SAMPLES_NUM = 7000


# text -- 16bit numbers
def write_file(file, text: list[int]):
    with open(file, 'wb') as f:
        for x in text:
            f.write(x.to_bytes(2, byteorder='little'))
        f.close()


def read_file(file):
    text = []
    with open(file, 'rb') as f:
        data = f.read()
        for pair in zip(data[::2], data[1::2]):
            el1, el2 = pair
            text.append((el2 << 8) | el1)
        f.close()
    return text


def get_plaintext_filename(variant: int, index: int, is_mutated: bool):
    return './key_check/pt_{0}_{1}{2}.bin'.format(variant, index, '_mut' if is_mutated else '')


def get_ciphertext_filename(variant: int, index: int, is_mutated: bool):
    return './key_check/ct_{0}_{1}{2}.bin'.format(variant, index, '_mut' if is_mutated else '')


def generate_ciphertext(alpha, samples_num, index):
    plaintext = [randrange(0, MAX_16BIT_NUM) for _ in range(0, samples_num)]
    mutated_plaintext = [text ^ alpha for text in plaintext]

    filename_pt_nonmut = get_plaintext_filename(VARIANT, index, False)
    filename_pt_mut = get_plaintext_filename(VARIANT, index, True)
    filename_ct_nonmut = get_ciphertext_filename(VARIANT, index, False)
    filename_ct_mut = get_ciphertext_filename(VARIANT, index, True)
    write_file(filename_pt_nonmut, plaintext)
    write_file(filename_pt_mut, mutated_plaintext)

    subprocess.Popen('./data/heys.bin e {0} {1} {2}'.format(VARIANT, filename_pt_nonmut, filename_ct_nonmut),
                     shell=True,
                     stdout=subprocess.DEVNULL, stderr=None).communicate(timeout=10)
    subprocess.Popen('./data/heys.bin e {0} {1} {2}'.format(VARIANT, filename_pt_mut, filename_ct_mut), shell=True,
                     stdout=subprocess.DEVNULL,
                     stderr=None).communicate(timeout=10)

    ciphertext = read_file(filename_ct_nonmut)
    ciphertext_mut = read_file(filename_ct_mut)
    return ciphertext, ciphertext_mut


def check_candidate(ct_nonmut, ct_mut, k, beta):
    success_rate = 0
    for (ct1, ct2) in zip(ct_nonmut, ct_mut):
        d1 = heys_obj.substitute(heys_obj.mix_words(ct1 ^ k), DEC)
        d2 = heys_obj.substitute(heys_obj.mix_words(ct2 ^ k), DEC)
        if d1 ^ d2 == beta:
            success_rate += 1
    return success_rate


# checking keys from 0..=((1<<16)-1)
keys = [(i, 0) for i in range(0, MAX_16BIT_NUM + 1)]

for i, ((alpha, beta), _) in enumerate(filtered_candidates):
    ct_nonmut, ct_mut = generate_ciphertext(alpha, SAMPLES_NUM, i)

    with Pool() as pool:
        success_rates = list(pool.map(lambda k: check_candidate(ct_nonmut, ct_mut, k[0], beta), keys))

    max_rate = max(success_rates)
    key_list_with_max_rate = []
    for j, (k, _) in enumerate(keys):
        if success_rates[j] == max_rate:
            key_list_with_max_rate.append((k, success_rates[j]))
    keys = key_list_with_max_rate

    print("\n\n Iteration: {0}, keys_hex: {1}, keys: {2}, max key rate: {3}\n\n".format(i,
                                                                                        get_hex_tuple_0(
                                                                                            keys),
                                                                                        keys,
                                                                                        max_rate))
    # print("\n\n Iteration: {0}, keys_hex: {1}, keys: {2}, max key rate: {3}, success_rates: {4} \n\n".format(i,get_hex_tuple_0(keys),keys,max_rate,success_rates))

    # if there is only one key left, no sense in iterating further
    if len(keys) == 1:
        break



 Iteration: 0, keys_hex: [0x1013, 0x101b, 0x1093, 0x109b, 0x1112, 0x111a, 0x1192, 0x119a, 0x1813, 0x181b, 0x1893, 0x189b, 0x1912, 0x191a, 0x1992, 0x199a, 0x2633, 0x263b, 0x26b3, 0x26bb, 0x2e33, 0x2e3b, 0x2eb3, 0x2ebb, 0x6277, 0x627f, 0x62f7, 0x62ff, 0x6a77, 0x6a7f, 0x6af7, 0x6aff, 0x9013, 0x901b, 0x9093, 0x909b, 0x9112, 0x911a, 0x9192, 0x919a, 0x9813, 0x981b, 0x9893, 0x989b, 0x9912, 0x991a, 0x9992, 0x999a, 0xa633, 0xa63b, 0xa6b3, 0xa6bb, 0xae33, 0xae3b, 0xaeb3, 0xaebb, 0xe277, 0xe27f, 0xe2f7, 0xe2ff, 0xea77, 0xea7f, 0xeaf7, 0xeaff], keys: [(4115, 3), (4123, 3), (4243, 3), (4251, 3), (4370, 3), (4378, 3), (4498, 3), (4506, 3), (6163, 3), (6171, 3), (6291, 3), (6299, 3), (6418, 3), (6426, 3), (6546, 3), (6554, 3), (9779, 3), (9787, 3), (9907, 3), (9915, 3), (11827, 3), (11835, 3), (11955, 3), (11963, 3), (25207, 3), (25215, 3), (25335, 3), (25343, 3), (27255, 3), (27263, 3), (27383, 3), (27391, 3), (36883, 3), (36891, 3), (37011, 3), (37019, 3), (37138, 3), (37146, 3), (37266, 3), (372

In [ ]:
висновки